In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import time
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras import Sequential
from sklearn.metrics import classification_report
from keras.layers import Conv2D,Flatten,MaxPool2D,BatchNormalization,Activation,Dropout
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image
import csv
import mediapipe as mp

In [5]:
df = pd.read_csv("landmark2.csv",on_bad_lines='skip')
X = df.drop(df.columns[0], axis = 1)
y = df[df.columns[0]]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

len(y_train.unique())

#y_train=keras.utils.to_categorical(y_train,27)
#y_test=keras.utils.to_categorical(y_test)

27

In [7]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(units=63, input_shape=[63]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=27, activation='softmax')
])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 63)                4032      
                                                                 
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8192      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0

In [8]:
y_train_encoded = pd.get_dummies(y_train, prefix='target')
y_test_encoded = pd.get_dummies(y_test, prefix='target')
y_train_encoded.head(3)

,target_A,target_B,target_C,target_D,target_E,target_F,target_G,target_H,target_I,target_K,...,target_S,target_T,target_U,target_V,target_W,target_X,target_Y,target_d,target_n,target_s
39877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5907,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41920,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [10]:
model1.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [11]:
start = time.time()

history = model1.fit(
    X_train,
    y_train_encoded,
    validation_split =0.2,
    epochs=5,
    callbacks=[early_stop]
)

stop1 = time.time()


y_pred = model1.predict(X_test)

stop2 = time.time()
apprentissage_RFC = round(stop1-start,3)
prediction_RFC = round(stop2-start,3)
print(f"Temps d'apprentissage : {apprentissage_RFC} s")
print(f"Temps d'apprentissage+prediction : {prediction_RFC} s")

Epoch 1/5
1155/1155 [==============================] - 9s 6ms/step - loss: 0.9859 - accuracy: 0.6551 - val_loss: 0.2756 - val_accuracy: 0.9000
Epoch 2/5
1155/1155 [==============================] - 8s 7ms/step - loss: 0.3714 - accuracy: 0.8720 - val_loss: 0.3262 - val_accuracy: 0.8732
Epoch 3/5
1155/1155 [==============================] - 9s 8ms/step - loss: 0.2828 - accuracy: 0.9086 - val_loss: 0.1682 - val_accuracy: 0.9363
Epoch 4/5
1155/1155 [==============================] - 9s 8ms/step - loss: 0.2506 - accuracy: 0.9208 - val_loss: 0.1247 - val_accuracy: 0.9688
Epoch 5/5
482/482 [==============================] - 1s 2ms/step
Temps d'apprentissage : 42.973 s
Temps d'apprentissage+prediction : 44.451 s


In [12]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [13]:
liste_lettre=['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W',"X","Y"]
liste_nb = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

# Create a dictionary that maps letters to numbers
key = {nb: lettre for nb, lettre in zip(liste_nb, liste_lettre)}

# Print the dictionary
print(key)

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P', 15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U', 20: 'V', 21: 'W', 22: 'X', 23: 'Y'}


In [14]:
import cv2

# Ouvrir la caméra de l'ordinateur
camera = cv2.VideoCapture(0)

# Prendre une photo
ret, frame = camera.read()

# Sauvegarder la photo
cv2.imwrite('mon_image.jpg', frame)

# Fermer la caméra
camera.release()

image = Image.open('mon_image.jpg')



In [19]:
# For static images:
IMAGE_FILES = [image]
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    for idx, file in enumerate(IMAGE_FILES):
        # Read an image, flip it around y-axis for correct handedness output (see
        # above).
        image = cv2.flip(cv2.imread(file), 1)
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
        # Print handedness and draw hand landmarks on the image.
        print('Handedness:', results.multi_handedness)
        if not results.multi_hand_landmarks:
          continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
            coord = []
            for i in range(21):
                
                x = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x
                y = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y
                z = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z
                coord.append([x, y, z])
            coord_rs = tf.reshape(coord, [1, -1])
            live_pred = model1.predict(cood_rs)
            live_pred = np.argmax(live_pred, axis=1)
            display = key.get(int(live_pred))
            print(display)
        
            mp_drawing.draw_landmarks(
                annotated_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            cv2.imwrite(
                '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
            # Draw hand world landmarks.
            if not results.multi_hand_world_landmarks:
              continue
        for hand_world_landmarks in results.multi_hand_world_landmarks:
          mp_drawing.plot_landmarks(
            hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)


TypeError: Can't convert object to 'str' for 'filename'

In [42]:
coord_rs = tf.reshape(coord, [1, -1])
live_pred = model1.predict(coord_rs)
live_pred = np.argmax(live_pred, axis=1)
print(live_pred)

1/1 [==============================] - 0s 22ms/step
[16]
